<p style="font-weight:bold;"> <span style="font-size: 36px"> Equality Comparers </span> </p>

In [0]:
#!import "../DataModel/DataStructure"
using System;

SequenceEqual static extension method is a utility for double arrays to enable comparison with a pre-defined precision.

In [0]:
public static bool SequenceEqual(this double[] defaultArray, double[] testArray, double precision)
{
    if ( defaultArray == null || testArray == null ) return false; 
    if (defaultArray.Length != testArray.Length) return false;
    for (int i = 0; i < defaultArray.Length; i++){
        if(Math.Abs(defaultArray[i]-testArray[i]) >= precision) return false;
    }
    return true;
}

In [0]:
class RawVariableComparer: IEqualityComparer<RawVariable>
{
    private bool IgnoreValues;
    private RawVariableComparer(bool ignoreValues)
    {
        IgnoreValues = ignoreValues;
    }

    public bool Equals(RawVariable x, RawVariable y) =>
        x.AccidentYear == y.AccidentYear && x.AmountType == y.AmountType && x.DataNode == y.DataNode && x.AocType == y.AocType && 
        x.Novelty == y.Novelty && x.EstimateType == y.EstimateType && (IgnoreValues ? true : x.Values.SequenceEqual(y.Values, Precision));

    public int GetHashCode(RawVariable v) => 0;

    public static RawVariableComparer Instance(bool ignoreValues = false) => new RawVariableComparer(ignoreValues);
}

In [0]:
class IfrsVariableComparer: IEqualityComparer<IfrsVariable>
{
    private bool IgnoreValue;
    private IfrsVariableComparer(bool ignoreValue)
    {
        IgnoreValue = ignoreValue;
    }

    public bool Equals(IfrsVariable x, IfrsVariable y) =>
        x.AccidentYear == y.AccidentYear && x.AmountType == y.AmountType && x.DataNode == y.DataNode && x.AocType == y.AocType && 
        x.Novelty == y.Novelty && x.EstimateType == y.EstimateType && x.EconomicBasis == y.EconomicBasis && (IgnoreValue ? true : Math.Abs(x.Value - y.Value) < Precision); 

    public int GetHashCode(IfrsVariable v) => 0;

    public static IfrsVariableComparer Instance(bool ignoreValue = false) => new IfrsVariableComparer(ignoreValue);
}

In [0]:
class YieldCurveComparer: IEqualityComparer<YieldCurve>
{
    private YieldCurveComparer(){}

	public bool Equals(YieldCurve x, YieldCurve y) => 
        x.Year == y.Year && x.Month == y.Month && x.Scenario == y.Scenario && x.Currency == y.Currency && x.Id == y.Id && x.Values.SequenceEqual(y.Values, Precision);
	
	public int GetHashCode (YieldCurve x) => 0;

    public static YieldCurveComparer Instance() => new YieldCurveComparer();
}


In [0]:
using System;
using System.Collections.Generic;
using System.Diagnostics; 
using System.Linq;
using System.Linq.Expressions;
using System.Reflection;
using System.Threading;
//using Systemorph.Domain;
//using Systemorph.Utils.Reflection;
using static Systemorph.Vertex.Equality.IdentityPropertyExtensions;

In [0]:
class EqualityComparer<T> : IEqualityComparer<T>
{
    private static readonly System.Reflection.PropertyInfo[] IdentityProperties = typeof(T).GetIdentityProperties().ToArray();
    private static Func<T, T, bool> compiledEqualityFunction;

    private EqualityComparer() {
        compiledEqualityFunction = GetEqualityFunction();
    }

    public static readonly EqualityComparer<T> Instance = new EqualityComparer<T>();

    public bool Equals(T x, T y) => compiledEqualityFunction(x, y);
    public int GetHashCode(T obj) => 0;

    private static Func<T, T, bool> GetEqualityFunction()
    {
        var prm1 = Expression.Parameter(typeof(T));
        var prm2 = Expression.Parameter(typeof(T));

        // r1 == null && r2 == null
        var nullConditionExpression = Expression.AndAlso(Expression.Equal(prm1, Expression.Constant(null, typeof(T))), Expression.Equal(prm2, Expression.Constant(null, typeof(T))));
        // r1 != null && r2 != null
        var nonNullConditionExpression = Expression.AndAlso(Expression.NotEqual(prm1, Expression.Constant(null, typeof(T))), Expression.NotEqual(prm2, Expression.Constant(null, typeof(T))));
        // r1.prop1 == r2.prop1 && r1.prop2 == r2.prop2...... 
        var allPropertiesEqualExpression = IdentityProperties.Select(propertyInfo => Expression.Equal(Expression.Property(prm1, propertyInfo), Expression.Property(prm2, propertyInfo))).Aggregate(Expression.AndAlso);

        var equalityExpr = Expression.OrElse(nullConditionExpression, Expression.AndAlso(nonNullConditionExpression, allPropertiesEqualExpression));
        return Expression.Lambda<Func<T, T, bool>>(equalityExpr, prm1, prm2).Compile();
    }
}

In [0]:
public class IdentityPropertyReader<T> where T:BaseVariableIdentity {
    private static Func<T, string> identityReader;
    
    private IdentityPropertyReader(){
        identityReader = GetIdentityReader();
    }
    public static readonly IdentityPropertyReader<T> Instance = new IdentityPropertyReader<T>();

    private Func<T, string> GetIdentityReader(){
        var parameter = Expression.Parameter(typeof(RawVariable));
        var identityProperties = typeof(RawVariable).GetIdentityProperties().Select(x=>x.Name).Where(x=> x != "AocType" && x != "Novelty").ToArray();
        var expressionstToConcat = identityProperties.SelectMany(x=> new Expression[]{Expression.Constant(x+": "), Expression.Property(parameter, x) })
                                                     .ToArray();
        var partialConcatExpression = Expression.Call(typeof(IdentityPropertyReader<RawVariable>).GetMethod("concat"), expressionstToConcat[0], expressionstToConcat[1]);
        for (int i = 2; i < expressionstToConcat.Count(); i++){
            if(i%2==0) partialConcatExpression = Expression.Call(typeof(IdentityPropertyReader<RawVariable>).GetMethod("concat"), partialConcatExpression, Expression.Constant(", "));
            if(i%2==1 && identityProperties[(int)Math.Floor((double)i/2)] == "AccidentYear"){
                partialConcatExpression = Expression.Call(typeof(IdentityPropertyReader<RawVariable>).GetMethod("concat2"), partialConcatExpression, expressionstToConcat[i]);   
            } else{
                partialConcatExpression = Expression.Call(typeof(IdentityPropertyReader<RawVariable>).GetMethod("concat"), partialConcatExpression, expressionstToConcat[i]);   
            }
        } // i needed (imo) to do it like this, since int? gives problem as Object...
        return Expression.Lambda<Func<T,string>>(partialConcatExpression, parameter).Compile();
    }
    public string GetIdentityString(T x) => identityReader(x);
    
    public static string concat(Object? first, Object? second){
        if(first == null) first = "";
        if(second == null) second = "";
        return first.ToString()+second.ToString();
    }
    
    public static string concat2(Object? first, int? second){
        if(first == null) first = "";
        if(second == null) return first.ToString();
        return first.ToString()+second.ToString();
    }
}

In [0]:
var a = new RawVariable(){AccidentYear = null, AocType = "CL", Novelty = "C", DataNode = "GIC1.1", AmountType ="CL", EstimateType="BE"};

In [0]:
var reader2 = IdentityPropertyReader<RawVariable>.Instance;

In [0]:
reader2.GetIdentityString(a)